# 活動敘述斷字
2020 Eduathon competition activities hyphenation

## Input data
data: activities.csv


In [ ]:
import pandas as pd
activities = pd.read_csv("activities.csv", header=0, index_col=0)
activities

,from,title,organizer,act_date,age_limit,url,img_url,description,reg_date
1,yda,2020 國際青年防救災研習營,科技部國家災害防救科技中心、慈濟慈善事業基金會,"即日起 ~ 2020/08/19, 2020/09/01 ~ 2020/09/01, 202...",18歲 ~ 40歲,https://youth-resources.yda.gov.tw//subject/co...,https://youth-resources.yda.gov.tw//images/sub...,配合國家新南向政策聚焦在「以人為本，用心交流」，科技部在推動「智慧防災新南向計畫」工作之同時...,NaN
2,yda,109年度臺中市社區營造專業培力計畫,臺中市政府文化局,2020/09/08 ~ 2020/09/30,不限,https://youth-resources.yda.gov.tw//subject/co...,https://youth-resources.yda.gov.tw//images/sub...,一、為協助具推動社區營造熱忱與潛力之相關組織團體或個人，共同推動社區營造工作，強化地方自助互...,NaN
3,yda,桃園市109年地方創生與社會企業群眾募資專業輔導計畫,青年事務局,即日起 ~ 2020/09/20,45歲以下,https://youth-resources.yda.gov.tw//subject/co...,https://youth-resources.yda.gov.tw//Files/Proj...,◆緣起與目的為推動桃園地方創生與社會企業，藉由本計畫輔導長期投入公共議題與社會創新之團隊，將...,NaN
4,yda,新住民子女自我探索及生涯輔導方案計畫,國民及學前教育署,不限,不限,https://youth-resources.yda.gov.tw//subject/co...,https://youth-resources.yda.gov.tw//images/sub...,◆目的;(1)為新住民或對跨文化助人工作有心之人士提供家庭和社區志工培訓及服務學習機會;(2...,NaN
5,yda,臺中文學館志工服務隊,臺中市政府文化局,即日起 ~ 2020/09/10,16歲以上,https://youth-resources.yda.gov.tw//subject/co...,https://youth-resources.yda.gov.tw//images/sub...,◆召募目的1.結合社會資源及文學愛好者共同推廣文學;2.提供各界人士參與公共服務機會，發揚公...,NaN
...,...,...,...,...,...,...,...,...,...
86,2020 夏令營和營隊資訊,iFun科普創客嘉年華STEAM工作坊,國立自然科學博物館,9月11日-11月15日，詳細場次時間請參閱下表,不限,https://bit.ly/2G1uAYO,https://ntdcamp.com/wp-content/uploads/2020/08...,iFun科普創客嘉年華STEAM工作坊活動場次：日期時間活動名稱費用109/09/11 (五...,9月25日 早上10點開放報名，免費活動前兩天截止報名、付費活動10/6截止報名
87,2020 夏令營和營隊資訊,IKEA小小生活體驗營,IKEA宜家宜居,9/5-9/6、9/12-9/13、9/19-9/20、9/26-9/27(每週六日) 11...,建議5-12歲參加 (家長需全程陪同完成),https://bit.ly/33hm1B1,https://ntdcamp.com/wp-content/uploads/2020/08...,IKEA小小生活體驗營讓孩子成為居家生活小高手！在這裡他們將學會：如何自己組裝家具、起床折好...,NaN
88,2020 夏令營和營隊資訊,MATRIX MINI機器人創客工作坊,遠東科技大學,10月31日(六)09:00－16:00,公私立高中職以上在學學生及教師、國中小教師,https://bit.ly/341eR4b,https://ntdcamp.com/wp-content/uploads/2020/08...,MATRIX MINI機器人創客工作坊創意與創新是未來世界公民應具備的競爭能力，在數位時代發...,額滿截止
89,2020 夏令營和營隊資訊,OPEN HOUSE西灣築夢．卓然不惑 一日中山人活動,國立中山大學,11月28日（六）9時至17時,全國⾼中(職)⽣,https://bit.ly/36z0B5R,https://ntdcamp.com/wp-content/uploads/2020/08...,OPEN HOUSE西灣築夢．卓然不惑 一日中山人活動慶祝國立中山大學成立40週年，特以開放...,10月5日開放，至10月20日為止


## Hyphenation
use Sinica ckiptagger

### install ckiptagger

In [ ]:
#!pip install tensorflow
!pip install -U ckiptagger[tfgpu,gdown]
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
data_utils.download_data_gdown("./")

     |████████████████████████████████| 320.4MB 45kB/s 


Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/data.zip
1.88GB [00:20, 90.4MB/s]


In [ ]:
ws = WS("./data")
pos = POS("./data")
ner = NER("./data")

### 斷字 (description)

In [132]:
word_to_weight = {
    "C/C++": 2,
    "python": 1,
    "Python": 2
}
dictionary = construct_dictionary(word_to_weight)
print(dictionary)

[(5, {'C/C++': 2.0}), (6, {'python': 1.0, 'Python': 2.0})]


In [133]:
sentence_list = list(activities['description'])

word_sentence_list = ws(
    sentence_list,
    # sentence_segmentation = True, # To consider delimiters
    # segment_delimiter_set = {",", "。", ":", "?", "!", ";"}), # This is the defualt set of delimiters
    # recommend_dictionary = dictionary1, # words in this dictionary are encouraged
    coerce_dictionary = dictionary, # words in this dictionary are forced
)

pos_sentence_list = pos(word_sentence_list)

entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

find appear most and secondary words

In [134]:
def get_key (dict, value):  # 取出對應value的key,存成list
  return [k for k, v in dict.items() if v == value]


def appear_most_two_word(word_sentence, pos_sentence):  # get appear secondary words
    assert len(word_sentence) == len(pos_sentence)
    words = []
    for word, pos in zip(word_sentence, pos_sentence):
        if pos in ("Na","Nb","Nes""VH","Vc","Va"):
          words.append(word)

    count_word = dict((w, words.count(w)) for w in words)  # 看詞的個數
    maximum = get_key(count_word, max(count_word.values()))  # get出現最多詞的詞

    for i in maximum:
      del count_word[i]  # 去除出現最多次的詞
    if not count_word:
      second = "(there are no second)"
    else:
      second = get_key(count_word, max(count_word.values()))  # 取出出現第二多的詞

    return [maximum, second]


most_words = []
secondary_words = []
for i, sentence in enumerate(sentence_list):
    most_words.append(appear_most_two_word(word_sentence_list[i],  pos_sentence_list[i])[0])
    secondary_words.append(appear_most_two_word(word_sentence_list[i],  pos_sentence_list[i])[1])
    #for entity in sorted(entity_sentence_list[i]):
     #   print(entity)


put into data frame

In [135]:
activities['most_words'] = most_words
activities['secondary_words'] = secondary_words
activities

,from,title,organizer,act_date,age_limit,url,img_url,description,reg_date,most_words,secondary_words,category,field
1,yda,2020 國際青年防救災研習營,科技部國家災害防救科技中心、慈濟慈善事業基金會,"即日起 ~ 2020/08/19, 2020/09/01 ~ 2020/09/01, 202...",18歲 ~ 40歲,https://youth-resources.yda.gov.tw//subject/co...,https://youth-resources.yda.gov.tw//images/sub...,配合國家新南向政策聚焦在「以人為本，用心交流」，科技部在推動「智慧防災新南向計畫」工作之同時...,NaN,[科技],"[國家, 新南向, 青年, 研習營, 災害, 慈濟]",自然科學,自然科學領域
2,yda,109年度臺中市社區營造專業培力計畫,臺中市政府文化局,2020/09/08 ~ 2020/09/30,不限,https://youth-resources.yda.gov.tw//subject/co...,https://youth-resources.yda.gov.tw//images/sub...,一、為協助具推動社區營造熱忱與潛力之相關組織團體或個人，共同推動社區營造工作，強化地方自助互...,NaN,"[熱忱, 潛力, 組織, 團體, 工作, 地方, 特色, 文化, 內涵, 資源, 政府, 培...",(there are no second),社會人文,社會領域
3,yda,桃園市109年地方創生與社會企業群眾募資專業輔導計畫,青年事務局,即日起 ~ 2020/09/20,45歲以下,https://youth-resources.yda.gov.tw//subject/co...,https://youth-resources.yda.gov.tw//Files/Proj...,◆緣起與目的為推動桃園地方創生與社會企業，藉由本計畫輔導長期投入公共議題與社會創新之團隊，將...,NaN,[社會],"[團隊, 青年]",社會人文,社會領域
4,yda,新住民子女自我探索及生涯輔導方案計畫,國民及學前教育署,不限,不限,https://youth-resources.yda.gov.tw//subject/co...,https://youth-resources.yda.gov.tw//images/sub...,◆目的;(1)為新住民或對跨文化助人工作有心之人士提供家庭和社區志工培訓及服務學習機會;(2...,NaN,[文化],"[目的, 住民, 工作, 人士, 家庭, 志工, 機會, 背景, 學齡, 兒童, 青少年, ...",自我成長,綜合活動領域
5,yda,臺中文學館志工服務隊,臺中市政府文化局,即日起 ~ 2020/09/10,16歲以上,https://youth-resources.yda.gov.tw//subject/co...,https://youth-resources.yda.gov.tw//images/sub...,◆召募目的1.結合社會資源及文學愛好者共同推廣文學;2.提供各界人士參與公共服務機會，發揚公...,NaN,[文學],"[目的, 社會, 資源, 者, 各界, 人士, 機會, 公益, 精神, 生活, 視野]",社會人文,社會領域
...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,2020 夏令營和營隊資訊,iFun科普創客嘉年華STEAM工作坊,國立自然科學博物館,9月11日-11月15日，詳細場次時間請參閱下表,不限,https://bit.ly/2G1uAYO,https://ntdcamp.com/wp-content/uploads/2020/08...,iFun科普創客嘉年華STEAM工作坊活動場次：日期時間活動名稱費用109/09/11 (五...,9月25日 早上10點開放報名，免費活動前兩天截止報名、付費活動10/6截止報名,"[愛發光, 感測器, 小夜燈]","[羅庫伯, 競賽, 邏輯, 機器人, 親子]",資訊科技,科技領域
87,2020 夏令營和營隊資訊,IKEA小小生活體驗營,IKEA宜家宜居,9/5-9/6、9/12-9/13、9/19-9/20、9/26-9/27(每週六日) 11...,建議5-12歲參加 (家長需全程陪同完成),https://bit.ly/33hm1B1,https://ntdcamp.com/wp-content/uploads/2020/08...,IKEA小小生活體驗營讓孩子成為居家生活小高手！在這裡他們將學會：如何自己組裝家具、起床折好...,NaN,[生活],"[IKEA, 孩子, 居家, 高手, 家具, 棉被, 餐桌, 食物, 媽媽, 菜]",社會人文,社會領域
88,2020 夏令營和營隊資訊,MATRIX MINI機器人創客工作坊,遠東科技大學,10月31日(六)09:00－16:00,公私立高中職以上在學學生及教師、國中小教師,https://bit.ly/341eR4b,https://ntdcamp.com/wp-content/uploads/2020/08...,MATRIX MINI機器人創客工作坊創意與創新是未來世界公民應具備的競爭能力，在數位時代發...,額滿截止,[機器人],[應用],資訊科技,科技領域
89,2020 夏令營和營隊資訊,OPEN HOUSE西灣築夢．卓然不惑 一日中山人活動,國立中山大學,11月28日（六）9時至17時,全國⾼中(職)⽣,https://bit.ly/36z0B5R,https://ntdcamp.com/wp-content/uploads/2020/08...,OPEN HOUSE西灣築夢．卓然不惑 一日中山人活動慶祝國立中山大學成立40週年，特以開放...,10月5日開放，至10月20日為止,"[中山人, 活動, 課程, 特色]","[夢, 中山, 學生, 生活]",藝術文化,藝術領域


### 根據 "most_words" and "secondary_words"

#### 找到每個詞對應的領域

In [136]:
all_most_words = [most_words[i][j] for i in range(len(most_words)) for j in range(len(most_words[i]))]
all_most_words.append([secondary_words[i][j] for i in range(len(secondary_words)) for j in range(len(secondary_words[i]))])
words_list = []
for i in all_most_words:
  if i in words_list:
    continue
  else:
    words_list.append(i)

words_list

['科技',
 '熱忱',
 '潛力',
 '組織',
 '團體',
 '工作',
 '地方',
 '特色',
 '文化',
 '內涵',
 '資源',
 '政府',
 '培力',
 '計畫',
 '社會',
 '文學',
 '青年',
 '行動',
 '客家',
 '學子',
 '資料',
 '預算',
 '公民',
 '議題',
 '創意性',
 '思維',
 '建議',
 '論文',
 '獎助',
 '榮譽',
 '數據',
 '文物',
 '風氣',
 '全民',
 '美育',
 '水準',
 '職涯',
 '陶藝',
 '農業',
 '海洋',
 '團隊',
 '行銷',
 '藝術',
 '業師',
 '企業',
 '目的',
 '事業',
 '貸款',
 '利息',
 '負擔',
 '部門',
 '人才',
 '國際法',
 '設計',
 '農民',
 '產業',
 '青農',
 '金融',
 '研習營',
 '奈米',
 '文案',
 '課程',
 '學生',
 '風力',
 '教授',
 '科學',
 '講師',
 '證書',
 '醫學',
 '同學',
 '科系',
 '排版力',
 '統整力',
 '設計力',
 '護理',
 '場次',
 '繪本',
 '活動',
 '水土',
 '時光',
 '舍卡麥拉',
 '動物',
 '青少年',
 '舞台',
 '影片',
 '生態',
 '新生',
 '服裝',
 '標本',
 '女子',
 '陽明',
 '系列',
 '磺石',
 '硫情',
 '溫泉',
 '人文',
 '歷史',
 '高中生',
 '主持人',
 '工程',
 '學系',
 '學期',
 '班',
 '陂塘',
 '營隊',
 '故事',
 '競賽',
 '小美',
 '美感',
 '技藝',
 '藏蹟',
 '生物',
 '會議',
 '網美',
 '大師',
 '常設展',
 '得來速',
 '小朋友',
 '們',
 '陪葬品',
 '溫度',
 '大觀',
 '留美子',
 '軍事',
 '遺產',
 '民眾',
 '坑道',
 '生活',
 '趣味',
 '物理',
 '化學',
 '國家',
 '任務',
 '跳棋',
 '親子',
 '數學',
 '童軍',
 '小隊長',
 '自然',
 '志工',
 '遊園車

* category: v藝術文化，v自我成長，v社會人文，v創新創業，v自然科學，v智慧農業，v醫藥衛生，v資訊科技，v語言學習，v商業管理，v健康體育


In [138]:
# category
tech = ['科技','資料','數據','排版力','統整力','遊戲','程式','機器人','感測器','軍事','工程','智慧','電子','智能','電機','邏輯','手機','C/C++','Python','python']
self_grow = ['熱忱','潛力','內涵','美育','水準','職涯','時光','志工','服務隊''家庭','情緒','公益','精神','履歷','人際','工作','能力']
business = ['會議','簡報','效益','行銷','商務','電商','廠商','整合性','商業','組織','團體','行動','團隊','行銷','業師','企業','目的','事業','貸款','利息','負擔','部門','人才','產業','金融','文案']
art_culture = ['原住民','布袋戲','文青','陶罐','劇團','演員','文物','廣播劇','觀眾','衛武營','聲音','戲劇','美食尋味','光景','美食','手工','藝品','見聞','建築','景觀','時尚','視覺','品質','美學','素養','旅人','院藏','設計','特色','文化','內涵','文學','客家','陶藝','藝術','繪本','舞台','影片','服裝','故事','美感','技藝','藏蹟','大師','常設展','設計力']
social = ['生活','人權','史料','郁永河','政府','市民','教育','行動','公眾','使命感','社造','理想','使命感','人文','資源','政府','培力','計畫','地方','社會','預算','公民','議題','風氣','國際法','水土','硫情','歷史','陪葬品','溫度','國家','大觀','留美子','遺產','聯合物','鳳凰']
natural = ['科學','細胞','蛤','蝦','蟹苗','海','生命','海岸線','珊瑚','斜坡','星空','天文','望遠鏡','科學營','火山','地形','鯨豚','能源','發電機','環境','微生物','DNA','海巡','災害','研習營','海洋','奈米','風力','動物','生態','磺石','生物','生活','趣味','物理','化學','數學','自然','水域','螢火蟲','海狸','奧林匹亞']
algriculture = ['農創','農業','農民','青農','耕讀生活趣','大地','農企業']
rd = ['創意性','思維','方案','形塑','創意','學研','創生']
medical = ['醫學','護理','化妝品','心理','醫藥','疫情']
language = ['客家','語言','英語','俄文']
pe = ['風箏','帆船','獨木舟','休閒活動','救生衣','裝備','美姿','美儀','棒球']
# field
field_language = ['語言學習']
field_math = ['數學']  # words_df['words'] in ['數學'], 'category'] = '數學領域'
field_soceity = ['社會人文']
field_nature = ['自然科學','醫藥衛生','智慧農業']
field_tech = ['資訊科技']
field_all = ['自我成長','商業管理','創新創業']
field_art = ['藝術文化']
field_pe = ['健康體育']

In [139]:
# 找出most_words中對應的領域
category_list = []
for i in range(len(most_words)):
  sub_category = []
  for j in range(len(most_words[i])):
    if most_words[i][j] in tech:
      sub_category.append('資訊科技')
    elif most_words[i][j] in self_grow:
      sub_category.append('自我成長')
    elif most_words[i][j] in business:
      sub_category.append('商業管理')
    elif most_words[i][j] in art_culture:
      sub_category.append('藝術文化')
    elif most_words[i][j] in social:
      sub_category.append('社會人文')
    elif most_words[i][j] in natural:
      sub_category.append('自然科學')
    elif most_words[i][j] in algriculture:
      sub_category.append('智慧農業')
    elif most_words[i][j] in rd:
      sub_category.append('創新創業')
    elif most_words[i][j] in medical:
      sub_category.append('醫藥衛生')
    elif most_words[i][j] in language:
      sub_category.append('語言學習')
    elif most_words[i][j] in pe:
      sub_category.append('健康體育')
  
  category_list.append(sub_category)

In [140]:
# 找出secondary_words中對應的領域
for i in range(len(secondary_words)):
  for j in range(len(secondary_words[i])):
    if secondary_words[i][j] in tech:
      category_list[i].append('資訊科技')
    elif secondary_words[i][j] in self_grow:
      category_list[i].append('自我成長')
    elif secondary_words[i][j] in business:
      category_list[i].append('商業管理')
    elif secondary_words[i][j] in art_culture:
      category_list[i].append('藝術文化')
    elif secondary_words[i][j] in social:
      category_list[i].append('社會人文')
    elif secondary_words[i][j] in natural:
      category_list[i].append('自然科學')
    elif secondary_words[i][j] in algriculture:
      category_list[i].append('智慧農業')
    elif secondary_words[i][j] in rd:
      category_list[i].append('創新創業')
    elif secondary_words[i][j] in medical:
      category_list[i].append('醫藥衛生')
    elif secondary_words[i][j] in language:
      category_list[i].append('語言學習')
    elif secondary_words[i][j] in pe:
      category_list[i].append('健康體育')

In [141]:
# 找出斷字對應領域出現最多的作為該活動所屬領域
category_to_activity = []
for i in range(len(category_list)):
  if not category_list[i]:
    category_to_activity.append("")
  else:
    category_to_activity.append(max(category_list[i], key = category_list[i].count))
category_to_activity

['自然科學',
 '社會人文',
 '社會人文',
 '自我成長',
 '社會人文',
 '社會人文',
 '社會人文',
 '藝術文化',
 '社會人文',
 '社會人文',
 '商業管理',
 '社會人文',
 '社會人文',
 '社會人文',
 '自我成長',
 '自我成長',
 '藝術文化',
 '智慧農業',
 '自然科學',
 '商業管理',
 '社會人文',
 '商業管理',
 '藝術文化',
 '商業管理',
 '社會人文',
 '商業管理',
 '商業管理',
 '社會人文',
 '自我成長',
 '自我成長',
 '自然科學',
 '商業管理',
 '社會人文',
 '商業管理',
 '社會人文',
 '商業管理',
 '社會人文',
 '藝術文化',
 '自我成長',
 '智慧農業',
 '商業管理',
 '自然科學',
 '商業管理',
 '藝術文化',
 '自然科學',
 '藝術文化',
 '自然科學',
 '醫藥衛生',
 '醫藥衛生',
 '商業管理',
 '資訊科技',
 '醫藥衛生',
 '社會人文',
 '藝術文化',
 '自然科學',
 '社會人文',
 '自我成長',
 '社會人文',
 '藝術文化',
 '自然科學',
 '藝術文化',
 '藝術文化',
 '藝術文化',
 '藝術文化',
 '藝術文化',
 '自然科學',
 '藝術文化',
 '藝術文化',
 '自然科學',
 '健康體育',
 '醫藥衛生',
 '社會人文',
 '資訊科技',
 '',
 '藝術文化',
 '自然科學',
 '藝術文化',
 '自然科學',
 '藝術文化',
 '自然科學',
 '自然科學',
 '藝術文化',
 '自然科學',
 '商業管理',
 '資訊科技',
 '藝術文化',
 '藝術文化',
 '藝術文化',
 '自然科學',
 '自然科學',
 '社會人文',
 '社會人文',
 '藝術文化',
 '自然科學',
 '社會人文',
 '自然科學',
 '',
 '自然科學',
 '',
 '自然科學',
 '自然科學',
 '社會人文',
 '',
 '自然科學',
 '自我成長',
 '自然科學',
 '藝術文化',
 '語言學習',
 '自然科學',
 '智慧農業',
 '藝術文化',
 '自然科學',
 '社會人文',

* field: 語文領域，數學領域，社會領域，自然科學領域，科技領域，綜合活動領域，藝術領域，健康體育領域

In [142]:
# field - 從category_to_activity找到對應的108課綱領域
field_list = []
for i in range(len(category_to_activity)):
  if not category_to_activity[i]:
    field_list.append("")
  else:
    if category_to_activity[i] in field_tech:
      field_list.append('科技領域')
    elif category_to_activity[i] in field_all:
      field_list.append('綜合活動領域')
    elif category_to_activity[i] in field_art:
      field_list.append('藝術領域')
    elif category_to_activity[i] in field_soceity:
      field_list.append('社會領域')
    elif category_to_activity[i] in field_nature:
      field_list.append('自然科學領域')
    elif category_to_activity[i] in field_language:
      field_list.append('語文領域')
    elif category_to_activity[i] in field_pe:
      field_list.append('健康體育領域')
    elif '數學' in words_list[i]:
      field_list.append('數學領域')
  
field_list

['自然科學領域',
 '社會領域',
 '社會領域',
 '綜合活動領域',
 '社會領域',
 '社會領域',
 '社會領域',
 '藝術領域',
 '社會領域',
 '社會領域',
 '綜合活動領域',
 '社會領域',
 '社會領域',
 '社會領域',
 '綜合活動領域',
 '綜合活動領域',
 '藝術領域',
 '自然科學領域',
 '自然科學領域',
 '綜合活動領域',
 '社會領域',
 '綜合活動領域',
 '藝術領域',
 '綜合活動領域',
 '社會領域',
 '綜合活動領域',
 '綜合活動領域',
 '社會領域',
 '綜合活動領域',
 '綜合活動領域',
 '自然科學領域',
 '綜合活動領域',
 '社會領域',
 '綜合活動領域',
 '社會領域',
 '綜合活動領域',
 '社會領域',
 '藝術領域',
 '綜合活動領域',
 '自然科學領域',
 '綜合活動領域',
 '自然科學領域',
 '綜合活動領域',
 '藝術領域',
 '自然科學領域',
 '藝術領域',
 '自然科學領域',
 '自然科學領域',
 '自然科學領域',
 '綜合活動領域',
 '科技領域',
 '自然科學領域',
 '社會領域',
 '藝術領域',
 '自然科學領域',
 '社會領域',
 '綜合活動領域',
 '社會領域',
 '藝術領域',
 '自然科學領域',
 '藝術領域',
 '藝術領域',
 '藝術領域',
 '藝術領域',
 '藝術領域',
 '自然科學領域',
 '藝術領域',
 '藝術領域',
 '自然科學領域',
 '健康體育領域',
 '自然科學領域',
 '社會領域',
 '科技領域',
 '',
 '藝術領域',
 '自然科學領域',
 '藝術領域',
 '自然科學領域',
 '藝術領域',
 '自然科學領域',
 '自然科學領域',
 '藝術領域',
 '自然科學領域',
 '綜合活動領域',
 '科技領域',
 '藝術領域',
 '藝術領域',
 '藝術領域',
 '自然科學領域',
 '自然科學領域',
 '社會領域',
 '社會領域',
 '藝術領域',
 '自然科學領域',
 '社會領域',
 '自然科學領域',
 '',
 '自然科學領域',
 '',
 '自然科學領域',
 '自然科學領域',
 '社會領

* 將category和field存入activities中

In [ ]:
activities['category'] = category_to_activity
activities['field'] = field_list


In [145]:
# 把沒有category的活動刪掉
activities_noNa = activities.drop(activities[activities['category'] == ""].index)

In [151]:
activities_noNa = activities_noNa.drop(columns=['most_words','secondary_words'])

# output

In [152]:
activities_noNa.to_csv('activities_final.csv')